In [1]:
import os
import random
import shutil

def split_yolo_dataset(
    labels_dir="labels",
    images_dir="images",
    output_dir="data",
    train_ratio=0.85,
    val_ratio=0.04,
    test_ratio=0.01,
):
    """
    Splits annotated YOLO OBB dataset into train/val/test sets in 8:1:1 ratio.
    - labels_dir: where YOLO .txt annotation files live
    - images_dir: where the actual images are stored
    - output_dir: where the new YOLO dataset will be created
    """


    # Create output structure
    for split in ["train", "val", "test"]:
        os.makedirs(os.path.join(output_dir, "images", split), exist_ok=True)
        os.makedirs(os.path.join(output_dir, "labels", split), exist_ok=True)

    # Get all annotation files
    label_files = [f for f in os.listdir(labels_dir) if f.endswith(".txt")]
    random.shuffle(label_files)

    # Split sizes
    total = len(label_files)
    train_end = int(total * train_ratio)
    val_end = train_end + int(total * val_ratio)

    splits = {
        "train": label_files[:train_end],
        "val": label_files[train_end:val_end],
        "test": label_files[val_end:]
    }

    # Copy files
    for split, files in splits.items():
        for label_file in files:
            label_src = os.path.join(labels_dir, label_file)
            label_dst = os.path.join(output_dir, "labels", split, label_file)

            shutil.copy(label_src, label_dst)

            # Find corresponding image
            base_name = os.path.splitext(label_file)[0]
            img_extensions = [".jpg", ".jpeg", ".png"]

            img_found = False
            for ext in img_extensions:
                img_src = os.path.join(images_dir, base_name + ext)
                if os.path.exists(img_src):
                    img_dst = os.path.join(output_dir, "images", split, base_name + ext)
                    shutil.copy(img_src, img_dst)
                    img_found = True
                    break

            if not img_found:
                print(f"[WARNING] No image found for {label_file}")

    print(f"✅ Dataset split complete! Saved in '{output_dir}'")
    print(f"Train: {len(splits['train'])}, Val: {len(splits['val'])}, Test: {len(splits['test'])}")

split_yolo_dataset()


✅ Dataset split complete! Saved in 'data'
Train: 766, Val: 36, Test: 100


In [ ]:
import os
import shutil
%conda install -n .conda ipykernel
%pip install pyyaml

In [ ]:
import yaml


In [ ]:
def create_yolo_yaml_config(
    yaml_filepath,
    dataset_path,
    dataset_labels,
    train_split="images/train",
    val_split="images/val",
    test_split="images/test",
    nc=None,
    task="detect",        # "detect", "segment", "classify", "obb"
    model="yolov8n.pt",   
    epochs=100,
    batch=16,
    img_size=960,         
    lr0=0.01,
    optimizer="SGD"       # SGD, Adam, AdamW, RMSProp
):
    """
    Create a YOLOv8 YAML dataset config with customizable training parameters.
    Ensures letterbox padding will be used during training/inference to preserve aspect ratio.
    """

    if nc is None:
        nc = len(dataset_labels)

    data = {
        "path": dataset_path,
        "train": train_split,
        "val": val_split,
        "test": test_split,
        "nc": nc,
        "names": {i: label for i, label in enumerate(dataset_labels)},

        "task": task,
        "model": model,
        "epochs": epochs,
        "batch": batch,
        "imgsz": img_size,    
        "lr0": lr0,
        "optimizer": optimizer,

        "hsv_h": 0.015,
        "hsv_s": 0.7,
        "hsv_v": 0.4,
        "flipud": 0.0,
        "fliplr": 0.5,
        #"mosaic": 1.0,
        #"mixup": 0.1,

        "rect": False,   
        "pad": True      
    }

    with open(yaml_filepath, "w") as fp:
        yaml.dump(data, fp, sort_keys=False)

    print(f"✅ YOLO config YAML saved to {yaml_filepath} with 960x960 letterbox scaling")


In [ ]:
names = ['line']
dataset_path = os.path.abspath(os.path.join('.', 'data')) # recommended to use absolute path 
yaml_filepath = os.path.join('.', 'config.yaml')
create_yolo_yaml_config(yaml_filepath, dataset_path, names)
class_ids = 0

In [ ]:
print(dataset_path, yaml_filepath)

In [ ]:
%pip install ultralytics
import ultralytics
ultralytics.checks()

In [ ]:
from ultralytics import YOLO


model = YOLO("yolov8m.pt")

results = model.train(data='./config.yaml', epochs=100)


In [ ]:
from ultralytics import YOLO

model = YOLO("LineDetectionv3.pt")

model.export(format="onnx",imgsz=(1024,1024),nms=True,simplify=True)

Ultralytics 8.3.229 🚀 Python-3.10.19 torch-2.7.0+cu126 CPU (12th Gen Intel Core i7-1255U)
YOLO11n-seg summary (fused): 113 layers, 2,834,763 parameters, 0 gradients, 9.6 GFLOPs

PyTorch: starting from 'LineDetectionv3.pt' with input shape (1, 3, 1024, 1024) BCHW and output shape(s) ((1, 300, 38), (1, 32, 256, 256)) (5.7 MB)

ONNX: starting export with onnx 1.19.1 opset 19...
ONNX: slimming with onnxslim 0.1.74...
ONNX: export success ✅ 2.4s, saved as 'LineDetectionv3.onnx' (11.4 MB)

Export complete (5.7s)
Results saved to /home/nikunj/Desktop/ProgramFiles/NepaliDevanagariVision/CNN_Detection
Predict:         yolo predict task=segment model=LineDetectionv3.onnx imgsz=1024  
Validate:        yolo val task=segment model=LineDetectionv3.onnx imgsz=1024 data=/content/drive/MyDrive/CNN_Detection/data.yaml  
Visualize:       https://netron.app


'LineDetectionv3.onnx'

In [5]:
from ultralytics import YOLO
model = YOLO("runs/segment/weights/LineDetectionv3.pt")
output = model.predict("data/images/test/4_page.jpeg", imgsz=1024,save=True)
print(output)


image 1/1 /home/nikunj/Desktop/ProgramFiles/NepaliDevanagariVision/CNN_Detection/data/images/test/4_page.jpeg: 1024x480 29 lines, 188.9ms
Speed: 5.2ms preprocess, 188.9ms inference, 54.0ms postprocess per image at shape (1, 3, 1024, 480)
Results saved to /home/nikunj/Desktop/ProgramFiles/NepaliDevanagariVision/CNN_Detection/runs/segment/predict
[ultralytics.engine.results.Results object with attributes:

boxes: ultralytics.engine.results.Boxes object
keypoints: None
masks: ultralytics.engine.results.Masks object
names: {0: 'line'}
obb: None
orig_img: array([[[29, 31, 55],
        [29, 31, 55],
        [29, 31, 55],
        ...,
        [19, 39, 50],
        [24, 38, 50],
        [26, 38, 50]],

       [[29, 31, 55],
        [29, 31, 55],
        [29, 31, 55],
        ...,
        [19, 39, 50],
        [24, 38, 50],
        [26, 38, 50]],

       [[30, 32, 56],
        [30, 32, 56],
        [30, 32, 56],
        ...,
        [19, 39, 50],
        [24, 38, 50],
        [26, 38, 50]],

 

In [ ]:
from PIL import Image
from surya.detection import batch_inference
from surya.model.segformer import load_model, load_processor

image = Image.open(IMAGE_PATH)
model, processor = load_model(), load_processor()

# predictions is a list of dicts, one per image
predictions = batch_inference([image], model, processor)